# IMPORTS

In [1]:
import sys
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
sys.path.append(os.path.abspath(".."))       # for 'protonet_STOP_bddoia_modules' folder
sys.path.append(os.path.abspath("../.."))    # for 'data' folder
sys.path.append(os.path.abspath("../../..")) # for 'models' and 'datasets' folders

print(sys.path)

['/users-1/eleonora/reasoning-shortcuts/IXShort/shortcut_mitigation/bddoia/notebooks', '/users-1/eleonora/anaconda3/envs/r4rr/lib/python38.zip', '/users-1/eleonora/anaconda3/envs/r4rr/lib/python3.8', '/users-1/eleonora/anaconda3/envs/r4rr/lib/python3.8/lib-dynload', '', '/users-1/eleonora/.local/lib/python3.8/site-packages', '/users-1/eleonora/anaconda3/envs/r4rr/lib/python3.8/site-packages', '/users-1/eleonora/reasoning-shortcuts/IXShort/shortcut_mitigation/bddoia', '/users-1/eleonora/reasoning-shortcuts/IXShort/shortcut_mitigation', '/users-1/eleonora/reasoning-shortcuts/IXShort']


In [2]:
import torch

from models import get_model
from datasets import get_dataset
from collections import defaultdict
from baseline_modules.arguments import args_dpl 
from torch.utils.data import Dataset, DataLoader

# DATA LOADING

In [3]:
args = args_dpl
dataset = get_dataset(args)
n_images, c_split = dataset.get_split()

encoder, decoder = dataset.get_backbone()
model = get_model(args, encoder, decoder, n_images, c_split)
model.start_optim(args)
loss = model.get_loss(args)

print(dataset)
print("Using Dataset: ", dataset)
print("Using backbone: ", encoder)
print("Using Model: ", model)
print("Using Loss: ", loss)

unsup_train_loader, unsup_val_loader, unsup_test_loader = dataset.get_data_loaders(args=args)

Available datasets: ['mnmath', 'xor', 'clipboia', 'shortmnist', 'restrictedmnist', 'minikandinsky', 'presddoia', 'prekandinsky', 'sddoia', 'clipkandinsky', 'addmnist', 'clipshortmnist', 'boia_original', 'boia_original_embedded', 'clipsddoia', 'boia', 'kandinsky', 'halfmnist']
Available models: ['promnistltn', 'sddoiann', 'kandnn', 'sddoiadpl', 'sddoialtn', 'presddoiadpl', 'boiann', 'mnistclip', 'prokanddpl', 'promnistdpl', 'xornn', 'mnistnn', 'mnistslrec', 'kandpreprocess', 'kandsl', 'kandsloneembedding', 'prokandltn', 'kandcbm', 'prokandsl', 'boiacbm', 'kanddpl', 'kandltn', 'xorcbm', 'sddoiaclip', 'xordpl', 'sddoiacbm', 'mnistltnrec', 'mnmathcbm', 'mnmathdpl', 'kandclip', 'minikanddpl', 'mnistdpl', 'mnistltn', 'boiadpl', 'boialtn', 'prokandsloneembedding', 'mnistpcbmdpl', 'mnistcbm', 'probddoiadpl', 'mnistpcbmsl', 'mnistpcbmltn', 'mnistsl', 'mnistdplrec', 'cvae', 'cext', 'mnmathnn', 'promnistsl']
Using Dataset:  <datasets.boia_original_embedded.FasterBDDOIADataset object at 0x7f2f614e

## Distribution

In [7]:
attribute_counts = torch.zeros(21, dtype=torch.long).to('cuda:0')  # Assuming CUDA
total_samples = 0

for i, batch in enumerate(unsup_train_loader):
    attr_labels = torch.stack(batch['attr_labels']).to(attribute_counts.device)
    # Accumulate 1s per column
    attribute_counts += attr_labels.sum(dim=0).long()
    # Accumulate total samples
    total_samples += attr_labels.size(0)

attribute_stats = {
    f'Concept {i}': {
        'count_1s': int(count.item()),
        'percentage': round(100.0 * count.item() / total_samples, 2)
    }
    for i, count in enumerate(attribute_counts)
}

print(f"\nTotal samples processed: {total_samples}")
print("Attribute Label Statistics (number of 1s and percentage over total):")
for attr, stats in attribute_stats.items():
    print(f"{attr}: {stats['count_1s']} ({stats['percentage']}%)")



Total samples processed: 16080
Attribute Label Statistics (number of 1s and percentage over total):
Concept 0: 5478 (34.07%)
Concept 1: 2448 (15.22%)
Concept 2: 3422 (21.28%)
Concept 3: 3766 (23.42%)
Concept 4: 1095 (6.81%)
Concept 5: 171 (1.06%)
Concept 6: 116 (0.72%)
Concept 7: 3703 (23.03%)
Concept 8: 329 (2.05%)
Concept 9: 100 (0.62%)
Concept 10: 464 (2.89%)
Concept 11: 220 (1.37%)
Concept 12: 115 (0.72%)
Concept 13: 629 (3.91%)
Concept 14: 270 (1.68%)
Concept 15: 3182 (19.79%)
Concept 16: 3177 (19.76%)
Concept 17: 2587 (16.09%)
Concept 18: 4303 (26.76%)
Concept 19: 2796 (17.39%)
Concept 20: 1563 (9.72%)
